# **BAG OF WORDS EN PYSPARK**



## 1. Instalar pyspark

In [1]:
! pip install pyspark

     |████████████████████████████████| 281.3 MB 33 kB/s 
     |████████████████████████████████| 198 kB 55.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=3d8b2db22f425d03b9be63cbc84c8e216f46d616abef641f9fcfec7c090a36e3
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


## 2. Guardar la libería en una variable con SparkContext

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
sc = SparkContext()

#3. Importar librerías

In [36]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)


#4. Algoritmo bow

In [37]:
data = [
    ['a', 'Jan', 'John', 'This is a document'],
    ['b', 'Feb', 'Mary', 'A book by Mary'],
    ['c', 'Mar', 'Luke', 'Newspaper article'],
    ['d', 'Apr', 'Mark', None]
]
columns = ['Title', 'Month', 'Author', 'Document']
df = spark.createDataFrame(data, columns)
df.show()

+-----+-----+------+------------------+
|Title|Month|Author|          Document|
+-----+-----+------+------------------+
|    a|  Jan|  John|This is a document|
|    b|  Feb|  Mary|    A book by Mary|
|    c|  Mar|  Luke| Newspaper article|
|    d|  Apr|  Mark|              null|
+-----+-----+------+------------------+



In [38]:
bow0 = df.rdd\
    .filter(lambda x: x.Document)\
    .map( lambda x: x.Document.replace(',',' ').replace('.',' ').replace('-',' ').lower())\
    .flatMap(lambda x: x.split())\
    .map(lambda x: (x, 1))
bow0.reduceByKey(lambda x,y:x+y).take(50)

[('this', 1),
 ('is', 1),
 ('book', 1),
 ('mary', 1),
 ('newspaper', 1),
 ('a', 2),
 ('document', 1),
 ('by', 1),
 ('article', 1)]

In [41]:
from pyspark.sql.functions import explode, split, regexp_replace, col, lower
df.select(explode(split(regexp_replace("Document", "[,.-]", " "), "\s+")).alias("palabra"))\
    .groupby(lower(col("palabra")).alias("lower"))\
    .count()\
    .show()

+---------+-----+
|    lower|count|
+---------+-----+
| document|    1|
|       by|    1|
|     mary|    1|
|       is|    1|
|        a|    2|
|     this|    1|
|     book|    1|
|newspaper|    1|
|  article|    1|
+---------+-----+

